We check the decomposition of a multi-controlled phase gate with arbitrary $\theta$ obtained via ZX calculus. The decomposition has 1-norm equal to $3$, which is sub-optimal.

In [2]:
import numpy as np
from util import return_ptm

In [3]:
def apply_mcphase_theta(op: np.ndarray, params: dict):
    n = int(np.log2(op.shape[1]))
    if op.shape[1] != 2**n:
        raise ValueError("Matrix shape error: the matrix dimension must be a power of 2")
    theta = params["theta"]
    mcphase = np.identity(2**n, dtype=complex)
    mcphase[-1, -1] = np.exp(1j*theta)
    return mcphase@op@mcphase.conj().T 

def apply_mcphase_ancilla_measurement_y(op: np.ndarray, params=dict):
    n = int(np.log2(op.shape[1]))
    if op.shape[1] != 2**n:
        raise ValueError("Matrix shape error: the matrix dimension must be a power of 2")
    ancilla_state = 1/np.sqrt(2)*np.array([[1], [1]])
    rho_plus_i = 1/2*np.array([[1, -1j], [1j, 1]])
    rho_minus_i = 1/2*np.array([[1, 1j], [-1j, 1]])
    rho_ancilla_state = ancilla_state@ancilla_state.conj().T
    op_with_ancilla = np.kron(op, rho_ancilla_state)
    op_with_ancilla_mcz = apply_mcphase_theta(op_with_ancilla, {"theta": params["theta"]})
    proj_plus_i = np.kron(np.identity(2**n), rho_plus_i)
    proj_minus_i = np.kron(np.identity(2**n), rho_minus_i)
    op_proj_plus_i = proj_plus_i@op_with_ancilla_mcz@proj_plus_i
    op_proj_minus_i = proj_minus_i@op_with_ancilla_mcz@proj_minus_i 
    op_plus_i_trace = np.trace(op_proj_plus_i.reshape(2**n , 2, 2**n, 2), axis1=1, axis2=3)
    op_minus_i_trace = np.trace(op_proj_minus_i.reshape(2**n , 2, 2**n, 2), axis1=1, axis2=3)
    # op_trace = np.trace(op_with_ancilla_mcz.reshape(2**n , 2, 2**n, 2), axis1=1, axis2=3) # only for testing
    if params["result"] == 0:
        return op_plus_i_trace
    elif params["result"] == 1:
        return op_minus_i_trace

Now let us focus on the case $n=4$ and $m = m' = 2$

In [4]:
num_qubits = 4
m = 2
m_prime = num_qubits - m
theta = np.pi/6

In [5]:
target_ptm_mcphase = return_ptm(apply_mcphase_theta, num_qubits, params={"theta": theta})

Now we obtain the PTMs for each term in the decomposition
1. $Y$-term (requires classical communication)

In [6]:
deco_dict = {}
deco_dict["1"] = {}
deco_dict["1"]["q"] = 1
deco_dict["1"]["ptm"] = np.kron(return_ptm(apply_mcphase_theta, m, params={"theta": np.pi/2}), return_ptm(apply_mcphase_ancilla_measurement_y, m_prime, params={"theta": theta, "result": 0})) 
deco_dict["1"]["ptm"] += np.kron(return_ptm(apply_mcphase_theta, m, params={"theta": -np.pi/2}), return_ptm(apply_mcphase_ancilla_measurement_y, m_prime, params={"theta": theta, "result": 1}))

2. $\mathcal{E}_{\mathrm{MCZ}-\ket{\pm}_a}^{(m)} \otimes \mathcal{I}^{\otimes m'} $

In [7]:
def apply_mcphase_ancilla_measurement_x(op: np.ndarray, params=dict):
    n = int(np.log2(op.shape[1]))
    if op.shape[1] != 2**n:
        raise ValueError("Matrix shape error: the matrix dimension must be a power of 2")
    ancilla_state = 1/np.sqrt(2)*np.array([[1], [1]])
    rho_plus = 1/2*np.array([[1, 1], [1, 1]])
    rho_minus = 1/2*np.array([[1, -1], [-1, 1]])
    rho_ancilla_state = ancilla_state@ancilla_state.conj().T
    op_with_ancilla = np.kron(op, rho_ancilla_state)
    op_with_ancilla_mcz = apply_mcphase_theta(op_with_ancilla, {"theta": params["theta"]})
    proj_plus = np.kron(np.identity(2**n), rho_plus)
    proj_minus = np.kron(np.identity(2**n), rho_minus)
    op_proj_plus = proj_plus@op_with_ancilla_mcz@proj_plus
    op_proj_minus = proj_minus@op_with_ancilla_mcz@proj_minus 
    op_plus_trace = np.trace(op_proj_plus.reshape(2**n , 2, 2**n, 2), axis1=1, axis2=3)
    op_minus_trace = np.trace(op_proj_minus.reshape(2**n , 2, 2**n, 2), axis1=1, axis2=3)
    # op_trace = np.trace(op_with_ancilla_mcz.reshape(2**n , 2, 2**n, 2), axis1=1, axis2=3) # only for testing
    return op_plus_trace - op_minus_trace
    
    

In [8]:
deco_dict["2"] = {}
deco_dict["2"]["q"] = 1/2
deco_dict["2"]["ptm"] = np.kron(return_ptm(apply_mcphase_ancilla_measurement_x, m, params={"theta": np.pi}), return_ptm(apply_mcphase_theta, m_prime, params={"theta": 2*np.pi}))

3. $\mathcal{E}_{\mathrm{MCZ}-\ket{\pm}_a}^{(m)} \otimes \mathcal{MCP}^{( m')}(\theta) $

In [9]:
deco_dict["3"] = {} 
deco_dict["3"]["q"] = -1/2
deco_dict["3"]["ptm"] = np.kron(return_ptm(apply_mcphase_ancilla_measurement_x, m, params={"theta": np.pi}), return_ptm(apply_mcphase_theta, m_prime, params={"theta": theta}))

4. $\mathcal{I}^{\otimes m} \otimes \mathcal{E}_{\mathrm{MCP}(\theta)-\ket{\pm}_a}^{(m')}$

In [10]:
deco_dict["4"] = {}
deco_dict["4"]["q"] = 1/2
deco_dict["4"]["ptm"] = np.kron(return_ptm(apply_mcphase_theta, m, params={"theta": 2*np.pi}), return_ptm(apply_mcphase_ancilla_measurement_x, m_prime, params={"theta": theta}))

5.  $\mathcal{MCZ}^{(m)}\otimes \mathcal{E}_{\mathrm{MCP}(\theta)-\ket{\pm}_a}^{(m')} $

In [11]:
deco_dict["5"] = {}
deco_dict["5"]["q"] = -1/2
deco_dict["5"]["ptm"] =  np.kron(return_ptm(apply_mcphase_theta, m, params={"theta": np.pi}), return_ptm(apply_mcphase_ancilla_measurement_x, m_prime, params={"theta": theta}))

In [12]:
cut = sum([deco_dict[key]["q"]*deco_dict[key]["ptm"] for key in deco_dict.keys()])

In [13]:
np.max(np.abs(target_ptm_mcphase - cut))

np.float64(4.440892098500626e-16)

which shows that the decomposition is correct!